# Meta-analysis of antidepressant exposure GWAS

Conduct GWAS meta-analysis using [Hail](https://hail.is). Import GWAS sumstats in [GWAS VCF](https://github.com/MRCIEU/gwas-vcf-specification) format, which can be read into Hail and understood like any other VCF file. See also [meta-analysis functions from the Pan UK Biobank](https://github.com/atgu/ukbb_pan_ancestry/blob/master/utils/meta_analysis.py). 

In [1]:
import pandas as pd
import hail as hl
hl.init(local='local[8]')

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
SLF4J: Class path contains SLF4J bindings targeting slf4j-api versions 1.7.x or earlier.
SLF4J: Ignoring binding found at [jar:file:/opt/homebrew/anaconda3/envs/hail/lib/python3.10/site-packages/pyspark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See https://www.slf4j.org/codes.html#ignoredBindings for an explanation.
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.2
SparkUI available at http://sakyo.home:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.120-f00f916faf78
LOGGING: writing to /Users/mark/Work/antidep-gwas/hail-20230802-1016-0.2.120-f00f916faf78.log


Open all the sumstats tables

In [2]:
gw = hl.read_matrix_table("mt/N06A.mt")

In [3]:
list(gw.entry)

['ES', 'SE', 'LP', 'AF', 'SS', 'EZ', 'SI', 'NC', 'ID']

In [37]:
gw.s.split("-")

<ArrayExpression of type array<str>>

In [59]:
samples = gw.s.collect()
meta_df = pd.DataFrame(
    {
    "dataset": samples,
    "group": ["-".join(s.split('-')[1:3]) for s in samples]
    }
)
meta_t = hl.Table.from_pandas(samples_datasets_df, key="dataset")
meta_t.show()

,
dataset,group
str,str
"""BBJ-N06A-EAS""","""N06A-EAS"""
"""FinnGen-N06A-EUR""","""N06A-EUR"""
"""GenScot-N06A-EUR""","""N06A-EUR"""
"""GenScot-N06AA-EUR""","""N06AA-EUR"""
"""GenScot-N06AB-EUR""","""N06AB-EUR"""
"""UKB-N06A-AFR""","""N06A-AFR"""
"""UKB-N06A-AMR""","""N06A-AMR"""
"""UKB-N06A-EAS""","""N06A-EAS"""


Annotate sumstats columns with phenotype and cluster group

In [83]:
gw = gw.annotate_cols(analysis=meta_t[gw.s].group)
gw.analysis.show()

,
s,analysis
str,str
"""UKB-N06AA-MID""","""N06AA-MID"""
"""FinnGen-N06A-EUR""","""N06A-EUR"""
"""UKB-N06A-MID""","""N06A-MID"""
"""UKB-N06A-EAS""","""N06A-EAS"""
"""GenScot-N06AA-EUR""","""N06AA-EUR"""
"""UKB-N06AB-AFR""","""N06AB-AFR"""
"""UKB-N06AB-EUR""","""N06AB-EUR"""
"""GenScot-N06A-EUR""","""N06A-EUR"""


Meta-analysis weight for each sumstats entry (IVW)

In [84]:
# inverse variance weight for each estimate
# sample size weight for other data
mw = gw.annotate_entries(
    ivw=1 / gw.SE ** 2,
    ncw=1 / gw.NC
)

# effect weighted by inverse variance
mw = mw.annotate_entries(
    ES_w=mw.ES * mw.ivw,
    AF_w=mw.AF * mw.ncw,
    SI_w=mw.SI * mw.ncw
)

Meta-analysis aggregation for each cluster

In [117]:
ma = mw.group_cols_by(mw.analysis).aggregate(
    ES=(hl.agg.array_sum(mw.ES_w) / hl.agg.array_sum(mw.ivw)).first(),
    SE=hl.map(lambda ivw_sum: hl.sqrt(1 / ivw_sum), hl.agg.array_sum(mw.ivw)).first(),
    NC=hl.agg.array_sum(mw.NC).first(),
    SS=hl.agg.array_sum(mw.SS).first(),
    AF=(hl.agg.array_sum(mw.AF_w) / hl.agg.array_sum(mw.ncw)).first(),
    SI=(hl.agg.array_sum(mw.SI_w) / hl.agg.array_sum(mw.ncw)).first()
)

ma = ma.annotate_entries(
    PP=hl.pchisqtail(ma.ES ** 2 / ma.SE ** 2, df=1)
)

In [118]:
ma.PP.show()

,,,,
,,'N06A-AFR','N06A-AMR','N06A-EAS'
locus,alleles,PP,PP,PP
locus<GRCh38>,array<str>,float64,float64,float64
chr1:10177,"[""A"",""AC""]",8.84e-01,1.67e-01,2.91e-03
chr1:10511,"[""G"",""A""]",NA,NA,NA
chr1:10616,"[""CCGCCGTTGCAAAGGCGCGCCG"",""C""]",1.05e-02,NA,NA
chr1:11008,"[""C"",""G""]",9.41e-01,1.84e-01,4.52e-01
chr1:11012,"[""C"",""G""]",9.41e-01,1.84e-01,4.52e-01
chr1:13110,"[""G"",""A""]",9.37e-01,7.74e-01,9.84e-01
chr1:13116,"[""T"",""G""]",1.11e-01,9.46e-01,2.08e-01


In [125]:
analyses = pd.DataFrame({"analysis": ma.analysis.collect()})
analyses

,analysis
0,N06A-AFR
1,N06A-AMR
2,N06A-EAS
3,N06A-EUR
4,N06A-MID
5,N06A-SAS
6,N06AA-AFR
7,N06AA-EUR
8,N06AA-MID
9,N06AA-SAS


In [111]:
ma.filter_cols(ma.analysis == "N06A-EUR").make_table().flatten().export("meta/ivw-N06A-EUR.tsv.bgz")

2023-08-02 14:46:22.587 Hail: INFO: merging 63 files totalling 1.0G...+ 1) / 62]
2023-08-02 14:46:24.168 Hail: INFO: while writing:
    meta/ivw-N06A-EUR.tsv.bgz
  merge time: 1.578s


In [109]:
ma.filter_cols(ma.analysis == "N06A-EUR").make_table().flatten().show()

,,,,,,,,,,,
locus,alleles,rsid,qual,filters,info.AF,N06A-EUR.ES,N06A-EUR.SE,N06A-EUR.NC,N06A-EUR.SS,N06A-EUR.AF,N06A-EUR.SI
locus<GRCh38>,array<str>,str,float64,set<str>,array<float64>,float64,float64,int64,int64,float64,float64
chr1:10177,"[""A"",""AC""]","""rs201752861""",-1.00e+01,{},[4.76e-01],-1.75e-02,1.09e-02,54524,211593,3.99e-01,4.69e-01
chr1:10511,"[""G"",""A""]","""rs534229142""",-1.00e+01,{},[1.27e-03],-6.84e-02,1.54e-01,54524,211593,1.26e-03,4.27e-01
chr1:10616,"[""CCGCCGTTGCAAAGGCGCGCCG"",""C""]","""rs376342519""",-1.00e+01,{},[9.92e-01],4.17e-02,7.30e-02,54524,211593,9.94e-01,4.45e-01
chr1:11008,"[""C"",""G""]","""rs575272151""",-1.00e+01,{},[1.24e-01],5.78e-03,1.86e-02,54524,211593,8.58e-02,4.86e-01
chr1:11012,"[""C"",""G""]","""rs544419019""",-1.00e+01,{},[1.24e-01],5.78e-03,1.86e-02,54524,211593,8.58e-02,4.86e-01
chr1:13110,"[""G"",""A""]","""rs540538026""",-1.00e+01,{},[1.48e-02],-7.19e-03,2.49e-02,54524,211593,5.88e-02,3.90e-01
chr1:13116,"[""T"",""G""]","""rs62635286""",-1.00e+01,{},[5.27e-02],-1.14e-03,1.46e-02,54524,211593,1.89e-01,4.03e-01
chr1:13118,"[""A"",""G""]","""rs62028691""",-1.00e+01,{},[5.27e-02],-1.14e-03,1.46e-02,54524,211593,1.89e-01,4.03e-01


In [80]:
ss = ma.make_table()
ss.show()

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,,,,,info,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
locus,alleles,rsid,qual,filters,AF,N06A-AFR.ES,N06A-AFR.SE,N06A-AFR.NC,N06A-AFR.SS,N06A-AFR.AF,N06A-AFR.SI,N06A-AMR.ES,N06A-AMR.SE,N06A-AMR.NC,N06A-AMR.SS,N06A-AMR.AF,N06A-AMR.SI,N06A-EAS.ES,N06A-EAS.SE,N06A-EAS.NC,N06A-EAS.SS,N06A-EAS.AF,N06A-EAS.SI,N06A-EUR.ES,N06A-EUR.SE,N06A-EUR.NC,N06A-EUR.SS,N06A-EUR.AF,N06A-EUR.SI,N06A-MID.ES,N06A-MID.SE,N06A-MID.NC,N06A-MID.SS,N06A-MID.AF,N06A-MID.SI,N06A-SAS.ES,N06A-SAS.SE,N06A-SAS.NC,N06A-SAS.SS,N06A-SAS.AF,N06A-SAS.SI,N06AA-AFR.ES,N06AA-AFR.SE,N06AA-AFR.NC,N06AA-AFR.SS,N06AA-AFR.AF,N06AA-AFR.SI,N06AA-EUR.ES,N06AA-EUR.SE,N06AA-EUR.NC,N06AA-EUR.SS,N06AA-EUR.AF,N06AA-EUR.SI,N06AA-MID.ES,N06AA-MID.SE,N06AA-MID.NC,N06AA-MID.SS,N06AA-MID.AF,N06AA-MID.SI,N06AA-SAS.ES,N06AA-SAS.SE,N06AA-SAS.NC,N06AA-SAS.SS,N06AA-SAS.AF,N06AA-SAS.SI,N06AB-AFR.ES,N06AB-AFR.SE,N06AB-AFR.NC,N06AB-AFR.SS,N06AB-AFR.AF,N06AB-AFR.SI,N06AB-EUR.ES,N06AB-EUR.SE,N06AB-EUR.NC,N06AB-EUR.SS,N06AB-EUR.AF,N06AB-EUR.SI,N06AB-MID.ES,N06AB-MID.SE,N06AB-MID.NC,N06AB-MID.SS,N06AB-MID.AF,N06AB-MID.SI,N06AB-SAS.ES,N06AB-SAS.SE,N06AB-SAS.NC,N06AB-SAS.SS,N06AB-SAS.AF,N06AB-SAS.SI,N06AX-AFR.ES,N06AX-AFR.SE,N06AX-AFR.NC,N06AX-AFR.SS,N06AX-AFR.AF,N06AX-AFR.SI,N06AX-EUR.ES,N06AX-EUR.SE,N06AX-EUR.NC,N06AX-EUR.SS,N06AX-EUR.AF,N06AX-EUR.SI,N06AX-SAS.ES,N06AX-SAS.SE,N06AX-SAS.NC,N06AX-SAS.SS,N06AX-SAS.AF,N06AX-SAS.SI
locus<GRCh38>,array<str>,str,float64,set<str>,array<float64>,array<float64>,array<float64>,array<int64>,array<int64>,array<float64>,array<float64>,array<float64>,array<float64>,array<int64>,array<int64>,array<float64>,array<float64>,array<float64>,array<float64>,array<int64>,array<int64>,array<float64>,array<float64>,array<float64>,array<float64>,array<int64>,array<int64>,array<float64>,array<float64>,array<float64>,array<float64>,array<int64>,array<int64>,array<float64>,array<float64>,array<float64>,array<float64>,array<int64>,array<int64>,array<float64>,array<float64>,array<float64>,array<float64>,array<int64>,array<int64>,array<float64>,array<float64>,array<float64>,array<float64>,array<int64>,array<int64>,array<float64>,array<float64>,array<float64>,array<float64>,array<int64>,array<int64>,array<float64>,array<float64>,array<float64>,array<float64>,array<int64>,array<int64>,array<float64>,array<float64>,array<float64>,array<float64>,array<int64>,array<int64>,array<float64>,array<float64>,array<float64>,array<float64>,array<int64>,array<int64>,array<float64>,array<float64>,array<float64>,array<float64>,array<int64>,array<int64>,array<float64>,array<float64>,array<float64>,array<float64>,array<int64>,array<int64>,array<float64>,array<float64>,array<float64>,array<float64>,array<int64>,array<int64>,array<float64>,array<float64>,array<float64>,array<float64>,array<int64>,array<int64>,array<float64>,array<float64>,array<float64>,array<float64>,array<int64>,array<int64>,array<float64>,array<float64>
chr1:10177,"[""A"",""AC""]","""rs201752861""",-1.00e+01,{},[4.76e-01],[1.25e-02],[8.52e-02],[730],[2734],[4.76e-01],[5.67e-01],[-4.13e-01],[2.99e-01],[81],[301],[4.05e-01],[4.69e-01],[-6.72e-01],[2.26e-01],[131],[981],[3.39e-01],[5.17e-01],[-1.75e-02],[1.09e-02],[54524],[211593],[3.99e-01],[4.69e-01],[1.74e-01],[2.35e-01],[155],[532],[4.28e-01],[4.21e-01],[-4.74e-02],[6.96e-02],[1479],[5027],[4.34e-01],[4.52e-01],[4.43e-02],[9.59e-02],[521],[2734],[4.76e-01],[5.67e-01],[-2.63e-02],[1.25e-02],[36034],[211593],[3.99e-01],[4.69e-01],[4.84e-03],[2.57e-01],[113],[532],[4.25e-01],[4.21e-01],[-3.33e-02],[7.69e-02],[1069],[5027],[4.34e-01],[4.52e-01],[-1.27e-02],[1.11e-01],[357],[2734],[4.78e-01],[5.67e-01],[-8.88e-03],[1.30e-02],[32332],[211593],[3.99e-01],[4.69e-01],[2.01e-01],[2.80e-01],[88],[532],[4.26e-01],[4.21e-01],[1.21e-01],[9.18e-02],[731],[5027],[4.32e-01],[4.52e-01],[2.40e-01],[1.87e-01],[115],[2734],[4.76e-01],[5.